# Loading Data to Snowflake

In [ ]:
import snowflake.connector
import pandas as pd
from snowflake.connector.pandas_tools import write_pandas

def connect_to_snowflake():
    """Establishes a connection to Snowflake using external browser authentication."""
    try:
        conn = snowflake.connector.connect(
            account='a206448_prod.us-east-1',
            user='rituja.das@thomsonreuters.com',
            role='A208177_PRODUCT_ANALYTICS_DATA_MDS_OWNER',
            authenticator='externalbrowser',
            warehouse='A208177_PRODUCT_ANALYTICS_DATA_MDS_WH',
            database='MYDATASPACE',
            schema='A208177_PRODUCT_ANALYTICS_DATA',
            client_session_keep_alive=True
        )
        print("Connected to Snowflake successfully")
        return conn
    except Exception as e:
        print(f"Failed to connect to Snowflake: {e}")
        return None

def create_table_in_snowflake(conn, df, table_name):
    """Generates and executes a CREATE TABLE statement in Snowflake based on DataFrame structure."""
    column_definitions = []
    
    for col in df.columns:
        dtype = str(df[col].dtype)
        
        if dtype == "object":
            column_definitions.append(f'"{col}" VARCHAR(500)')
        elif dtype == "float64":
            column_definitions.append(f'"{col}" FLOAT')
        elif dtype == "int64":
            column_definitions.append(f'"{col}" INT')
        elif "datetime" in dtype:
            column_definitions.append(f'"{col}" TIMESTAMP_NTZ')
        else:
            column_definitions.append(f'"{col}" VARCHAR(500)')
    
    column_definitions.append('"RUN_TIMESTAMP" TIMESTAMP_NTZ DEFAULT CURRENT_TIMESTAMP')
    
    create_table_sql = f"""
    CREATE OR REPLACE TABLE {table_name} (
        {', '.join(column_definitions)}
    );
    """
    
    try:
        cursor = conn.cursor()
        cursor.execute(create_table_sql)
        print(f"Table {table_name} created or replaced successfully.")
    except Exception as e:
        print(f"Error creating table: {e}")
    finally:
        cursor.close()

def truncate_table_in_snowflake(conn, table_name):
    """Truncates the Snowflake table to remove all existing data."""
    truncate_table_sql = f"TRUNCATE TABLE {table_name};"
    try:
        cursor = conn.cursor()
        cursor.execute(truncate_table_sql)
        print(f"Table {table_name} truncated successfully.")
    except Exception as e:
        print(f"Error truncating table: {e}")
    finally:
        cursor.close()

def insert_data_into_snowflake(conn, df, table_name):
    """Inserts data into Snowflake table in batches using write_pandas."""
    df['RUN_TIMESTAMP'] = pd.to_datetime(pd.Timestamp.now()).strftime('%Y-%m-%d %H:%M:%S.%f')
    
    try:
        success, num_chunks, num_rows, output = write_pandas(conn, df, table_name)
        print(f"Data successfully uploaded: {num_rows} rows inserted into {table_name}")
    except Exception as e:
        print(f"Error inserting data into Snowflake: {e}")

def upload_dataframe_to_snowflake(df, table_name):
    conn = connect_to_snowflake()
    if conn is None:
        return
    
    create_table_in_snowflake(conn, df, table_name)
    truncate_table_in_snowflake(conn, table_name)  # Clear existing data
    insert_data_into_snowflake(conn, df, table_name)
    conn.close()
    print("Snowflake connection closed.")


# Assuming df is your DataFrame
upload_dataframe_to_snowflake(df, "TABLE_NAME_INPUT_DATA_LOAD")

In [ ]:
def read_data_from_snowflake(table_name):
    # Establish a connection to Snowflake.
    conn = connect_to_snowflake()
    if conn is None:
        print("Connection to Snowflake failed.")
        return None

    # Create the SQL query to retrieve all records from the specified table.
    query = f"SELECT * FROM {table_name};"
    
    try:
        # Use Pandas to execute the SQL query and load the result into a DataFrame.
        df = pd.read_sql(query, conn)
        print(f"Data successfully retrieved from {table_name}")
        return df
    except Exception as e:
        # If there is an error during data retrieval, print an error message.
        print(f"Error reading data from Snowflake: {e}")
        return None
    finally:
        # Ensure the connection to Snowflake is closed.
        conn.close()


# Read data from the Snowflake table into a DataFrame.
df_snowflake = read_data_from_snowflake("TABLE_NAME_INPUT_DATA_LOAD")
df = df_snowflake